In [133]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import tensorflow as tf 
import pandas as pd 

In [134]:
from feedback_prize import utils

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [205]:
## Using a subset of our dataset 
X = pd.read_csv('../raw_data/train.csv',nrows=None)
X['discourse_type']=X['discourse_type'].str.replace('Concluding Statement','Concluding_Statement')

In [206]:
ID_TO_TEST='DBF7EB6A9E02'
#X = X.loc[X.id==ID_TO_TEST]
X.head(3)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75


In [207]:
## Creating features for labeling needs : 

    #Flag if the discourse is the same as the previous one
X['previous_discourse_flag']=np.where(X['discourse_type'].shift(1)==X['discourse_type'],1,0)

    #Get length of predictionstring
X['predictionstring_len'] = X['predictionstring'].apply(lambda txt:len(txt.split()))

In [208]:
X.head(2)

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,previous_discourse_flag,predictionstring_len
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,0,44
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,0,15


In [209]:
## Def a function for labelling discourses per word

def labelizer(label,len_,flag):
    
    if flag==0:
        label_first = f'B-{label} '
    else:
        label_first = f'I-{label} '
        
    return (label_first + f'I-{label} '*(len_-1)).strip()

In [210]:
# vectorize labelizer and apply to our df 
f = np.vectorize(labelizer)
X['label']=f(X['discourse_type'],X['predictionstring_len'],X['previous_discourse_flag'])
X.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,previous_discourse_flag,predictionstring_len,label
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,0,44,B-Lead I-Lead I-Lead I-Lead I-Lead I-Lead I-Le...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,0,15,B-Position I-Position I-Position I-Position I-...
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,0,16,B-Evidence I-Evidence I-Evidence I-Evidence I-...
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,1,63,I-Evidence I-Evidence I-Evidence I-Evidence I-...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,0,24,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...


In [211]:
## Groupby ID to get predictionstrings and labels as a unique string

X_grouped = X.groupby('id').agg({'predictionstring':' '.join,'label':' '.join})

## Transform into lists

X_grouped['label'] = X_grouped['label'].apply(lambda txt : txt.split())
X_grouped['predictionstring'] = X_grouped['predictionstring'].apply(lambda txt : txt.split())

X_grouped.head(3)

,predictionstring,label
id,,
0000D23A521A,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[B-Position, I-Position, I-Position, I-Positio..."
00066EA9880D,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
000E6DE9E817,"[2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, ...","[B-Position, I-Position, I-Position, I-Positio..."


In [218]:
labels_mapping = {'B-Lead' : 0,
                  'B-Position' : 1,
                  'B-Evidence' : 2,
                  'B-Claim' : 3,
                  'B-Concluding_Statement' : 4,
                  'B-Counterclaim' : 5,
                  'B-Rebuttal' : 6,
                  'I-Lead' : 7,
                  'I-Position' : 8,
                  'I-Evidence' : 9,
                  'I-Claim' : 10,
                  'I-Concluding_Statement' : 11,
                  'I-Counterclaim' : 12,
                  'I-Rebuttal': 13}


In [219]:
predictionstring = X_grouped.loc['00066EA9880D','predictionstring']
label = X_grouped.loc['00066EA9880D','label']
match = {p:labels_mapping[l] for p,l in zip(predictionstring,label)}
match

{'0': 0,
 '1': 7,
 '2': 7,
 '3': 7,
 '4': 7,
 '5': 7,
 '6': 7,
 '7': 7,
 '8': 7,
 '9': 7,
 '10': 7,
 '11': 7,
 '12': 7,
 '13': 7,
 '14': 7,
 '15': 7,
 '16': 7,
 '17': 7,
 '18': 7,
 '19': 7,
 '20': 7,
 '21': 7,
 '22': 7,
 '23': 7,
 '24': 7,
 '25': 7,
 '26': 7,
 '27': 7,
 '28': 7,
 '29': 7,
 '30': 7,
 '31': 7,
 '32': 7,
 '33': 7,
 '34': 7,
 '35': 7,
 '36': 7,
 '37': 7,
 '38': 7,
 '39': 7,
 '40': 7,
 '41': 7,
 '42': 7,
 '43': 7,
 '44': 7,
 '45': 7,
 '46': 7,
 '47': 7,
 '48': 7,
 '49': 7,
 '50': 7,
 '51': 7,
 '52': 7,
 '53': 7,
 '54': 7,
 '55': 7,
 '56': 7,
 '57': 7,
 '58': 7,
 '59': 7,
 '60': 7,
 '61': 7,
 '62': 7,
 '63': 7,
 '64': 7,
 '65': 7,
 '66': 7,
 '67': 7,
 '68': 7,
 '69': 7,
 '70': 7,
 '71': 7,
 '72': 7,
 '73': 7,
 '74': 7,
 '75': 7,
 '76': 7,
 '77': 1,
 '78': 8,
 '79': 8,
 '80': 8,
 '81': 8,
 '82': 8,
 '83': 8,
 '84': 8,
 '85': 8,
 '86': 8,
 '87': 8,
 '88': 8,
 '89': 8,
 '90': 8,
 '91': 8,
 '92': 8,
 '93': 8,
 '94': 8,
 '95': 8,
 '96': 8,
 '97': 8,
 '106': 3,
 '107': 10,
 '108':

In [220]:
essay = utils.get_essay('00066EA9880D')
tokens = tokenizer(essay,
                   return_attention_mask = True,
                   return_token_type_ids = False,
                   padding = 'max_length',
                   max_length = 1024,
                   truncation = True
                  )

In [471]:
def tokenize_labelize(essay,predictionstring,labels):
    
    tokens = tokenizer(essay,
                       return_attention_mask = True,
                       return_token_type_ids = False,
                       padding = 'max_length',
                       max_length = 1024,
                       truncation = True,
                       return_tensors='np'
                      )
    
    word_ids=tokens.word_ids()
    
    labels_mapping = {'B-Lead' : 0,
                  'B-Position' : 1,
                  'B-Evidence' : 2,
                  'B-Claim' : 3,
                  'B-Concluding_Statement' : 4,
                  'B-Counterclaim' : 5,
                  'B-Rebuttal' : 6,
                  'I-Lead' : 7,
                  'I-Position' : 8,
                  'I-Evidence' : 9,
                  'I-Claim' : 10,
                  'I-Concluding_Statement' : 11,
                  'I-Counterclaim' : 12,
                  'I-Rebuttal': 13}
    
    labels = [-100 if (w==None or w==word_ids[i-1]) else match.get(str(w),14) for i,w in enumerate(word_ids)]
    
        
    return {
    'input_ids' : tokens['input_ids'][0],
    'attention_mask' : tokens['attention_mask'][0],
    'labels': np.array(labels)
       }

In [472]:
essay = utils.get_essay('00066EA9880D')
predictionstring = X_grouped.loc['00066EA9880D']
labels = X_grouped.loc['00066EA9880D']
test = tokenize_labelize(essay,predictionstring,labels)

In [473]:
test

{'input_ids': array([ 101, 4062, 3238, ...,    0,    0,    0]),
 'attention_mask': array([1, 1, 1, ..., 0, 0, 0]),
 'labels': array([-100,    0, -100, ..., -100, -100, -100])}

In [474]:
## test to vectorize with small subset

X_test = X_grouped[0:5].reset_index()
getter_vectorized = np.vectorize(utils.get_essay) #vectorize utils
essays_vect = getter_vectorized(X_test['id']) #have an array of essays 
tokenize_labelize_vect = np.vectorize(tokenize_labelize) #vectorize func
result = tokenize_labelize_vect(essays_vect,X_test.predictionstring,X_test.label)

In [477]:
dataset = {'input_ids':[],
          'attention_mask':[],
          'labels':[]}


In [478]:
for r in result:
    dataset['input_ids'].append(r['input_ids'])
    dataset['attention_mask'].append(r['attention_mask'])
    dataset['labels'].append(r['labels'])

In [479]:
dataset

{'input_ids': [array([ 101, 2070, 2111, ...,    0,    0,    0]),
  array([ 101, 4062, 3238, ...,    0,    0,    0]),
  array([ 101, 6203, 1024, ...,    0,    0,    0]),
  array([ 101, 2052, 2017, ...,    0,    0,    0]),
  array([ 101, 1045, 2228, ...,    0,    0,    0])],
 'attention_mask': [array([1, 1, 1, ..., 0, 0, 0]),
  array([1, 1, 1, ..., 0, 0, 0]),
  array([1, 1, 1, ..., 0, 0, 0]),
  array([1, 1, 1, ..., 0, 0, 0]),
  array([1, 1, 1, ..., 0, 0, 0])],
 'labels': [array([-100,    0,    7, ..., -100, -100, -100]),
  array([-100,    0, -100, ..., -100, -100, -100]),
  array([-100,    0,    7, ..., -100, -100, -100]),
  array([-100,    0,    7, ..., -100, -100, -100]),
  array([-100,    0,    7, ..., -100, -100, -100])]}

In [480]:
tf.data.Dataset.from_tensor_slices(dataset)

<TensorSliceDataset shapes: {input_ids: (1024,), attention_mask: (1024,), labels: (1024,)}, types: {input_ids: tf.int64, attention_mask: tf.int64, labels: tf.int64}>

In [ ]:
## WORKFLOW 
# 1- tokenize an essay 
# 2- align the labels with the tokens 
# 3- get everything in a dictionnary 

In [6]:
## Creating tokens and visualize
tokens = tokenizer(text_1.split(),return_attention_mask=True,return_token_type_ids=False,
                  padding = 'max_length',max_length=45,truncation=True,is_split_into_words=True)


#pd.DataFrame({'text':[text_1.split()[pos[0]:pos[1]] for pos in tokens['offset_mapping']],'offset':tokens['offset_mapping']},index=pd.Index(tokens['input_ids']))

tokenizer.convert_ids_to_tokens(tokens['input_ids'])

['[CLS]',
 'this',
 'is',
 'a',
 'first',
 'test',
 '.',
 'the',
 'test',
 'contains',
 '3',
 'sentences',
 '.',
 'this',
 'is',
 'the',
 'last',
 'sentence',
 '.',
 'we',
 'have',
 'a',
 'second',
 'claim',
 'here',
 ',',
 'very',
 '##com',
 '##pl',
 '##ie',
 '##x',
 '##word',
 'so',
 'strange',
 '.',
 'and',
 'we',
 'finally',
 'have',
 'our',
 'wonderful',
 'conclusion',
 '[SEP]',
 '[PAD]',
 '[PAD]']

In [7]:
tokens.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 4,
 5,
 6,
 7,
 8,
 9,
 9,
 10,
 11,
 12,
 13,
 14,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 20,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 23,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 None,
 None,
 None]

In [9]:
## TODO 
## 3 TASKS 
## 1- Map the label to each word of a sentence 
## 2- Regroup the essay into one string ??
## 3 - Tokenize 
## 4 - Remap tokens to labels